In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
import warnings

import pandas_ta as ta
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("./train_adj.csv", index_col='date')
train = train.drop('Unnamed: 0', axis=1)
train = train.round(2)

In [14]:
train

,ticker,firm,volume,open,high,low,close,adjustTrue,SMA_10,SMA_20,SMA_50,SMA_100,SMA_200,EMA_10,EMA_20,EMA_50,EMA_100,EMA_200,RSI_14,ROC_10
date,,,,,,,,,,,,,,,,,,,,
2023-05-30,A000020,동화약품,201361.0,9960.0,10040.0,9640.0,9700.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-26,A000020,동화약품,196257.0,10050.0,10150.0,9850.0,9850.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-25,A000020,동화약품,398326.0,9660.0,10180.0,9660.0,10040.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-24,A000020,동화약품,205243.0,9770.0,9820.0,9550.0,9740.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-23,A000020,동화약품,641524.0,9160.0,9900.0,9160.0,9770.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-07,A383800,LX홀딩스,2714980.0,10550.0,11150.0,10500.0,10800.0,1,11090.0,11155.0,10878.0,10193.1,9941.50,11021.143378,11048.283770,10812.983746,10447.342607,10880.197413,44.775198,-3.571429
2021-06-04,A383800,LX홀딩스,1737593.0,10450.0,10650.0,10350.0,10450.0,1,11010.0,11125.0,10880.0,10208.9,9945.70,10917.299127,10991.304363,10798.749090,10447.395228,10875.916842,38.217870,-7.111111
2021-06-03,A383800,LX홀딩스,2709800.0,10650.0,10700.0,10300.0,10400.0,1,10930.0,11085.0,10879.0,10224.7,9949.45,10823.244740,10934.989662,10783.111870,10446.456709,10871.181351,37.375765,-7.142857


In [12]:
def calculate_technical_indicators(df: pd.DataFrame) -> pd.DataFrame:
    # Moving Averages
    df.ta.sma(close='Close', length=10, append=True)
    df.ta.sma(close='Close', length=20, append=True)
    df.ta.sma(close='Close', length=50, append=True)
    df.ta.sma(close='Close', length=100, append=True)
    df.ta.sma(close='Close', length=200, append=True)

    df.ta.ema(close='Close', length=10, append=True)
    df.ta.ema(close='Close', length=20, append=True)
    df.ta.ema(close='Close', length=50, append=True)
    df.ta.ema(close='Close', length=100, append=True)
    df.ta.ema(close='Close', length=200, append=True)

    # Momentum Indicators
    df.ta.rsi(close='Close', length=14, append=True)

    # MACD는 단기 EMA (12일)가 장기 EMA (26일)에 비해 얼마나 빠르게 움직이는지를 보여줍니다.
    df.ta.roc(close='Close', append=True)

    # Volume Indicators
    df.ta.vp(close='Close', volume='Volume', append=True)
    df.ta.obv(close='Close', volume='Volume', append=True)

    # Volatility Indicators
    df.ta.atr(close='Close', append=True)
    df.ta.bbands(close='Close', append=True)

    # Trend Strength Indicators
    df.ta.adx(close='Close', append=True)

    # Cycle Indicators
    df.ta.ht_trendline(close='Close', append=True)


    df.ta.efi(length=13, append=True) # Elder's Force Index (EFI): 알렉산더 엘더가 개발한 이 지표는 가격의 변동성과 거래량을 결합하여 주식의 '힘'을 측정합니다.
    df.ta.kama(length=10, append=True) # Kaufman's Adaptive Moving Average (KAMA): 이 지표는 변동성을 고려하여 보다 유연한 이동 평균을 제공합니다.
    df.ta.mfi(high='High', low='Low', close='Close', volume='Volume', length=14, append=True) # Money Flow Index (MFI): 이 지표는 가격과 거래량을 결합하여 주식이 과매수 또는 과매도 상태인지 판단합니다.
    df.ta.tii(length=14, append=True) # Trend Intensity Index (TII): 이 지표는 주식의 추세 강도를 측정하며, 방향과 강도를 동시에 판단할 수 있습니다.
    df.ta.vortex(high='High', low='Low', close='Close', length=14, append=True) # Vortex Indicator (VI): 이 지표는 최근 가격의 상승과 하락을 추적하여 상승 추세와 하락 추세를 식별합니다.

    return df


In [13]:
calculate_technical_indicators(train)

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    
    # 학습 데이터 생성
    train_close = train[train['종목코드'] == code][['일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)
    tc = train_close['종가']
    
    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측
    
    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    
    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)